# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [318]:
import pandas as pd
import numpy as np
import glob
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [319]:
# Set the folder path
guineaFolderPath = DATA_FOLDER + '/ebola/guinea_data'
liberiaFolderPath = DATA_FOLDER + '/ebola/liberia_data'
slFolderPath = DATA_FOLDER + '/ebola/sl_data'

#### Data preparation

In [320]:
#Year average
yearAverage = pd.DataFrame(columns=['country', 'new_cases', 'new_deaths'])

#get all data from the Guinea folder
#allGuineaFiles = glob.glob(guineaFolderPath + "/*.csv")
#guineaFram = pd.concat((pd.read_csv(f) for f in allGuineaFiles))

#### Needed data
We have to keep track of the date, the description and the total of case

We count as new case:
-  New cases of confirmed

We count as new death:
-  New deaths registered  
-  New deaths registered today (confirmed)

#### Guinea
Each file corresponding to one day

In [321]:
#Daily data
dailyData = pd.DataFrame(columns=['date', 'country', 'new_cases', 'new_deaths'])

for filename in glob.glob(guineaFolderPath + "/*.csv"):
    df = pd.read_csv(filename, index_col='Description')
    
    casesRow = df.loc["New cases of confirmed"]
    case = casesRow['Totals']
    
    try:
        deathRow = df.loc["New deaths registered today (confirmed)"]
    except KeyError:
        deathRow = df.loc["New deaths registered"]
    death = deathRow['Totals']
    
    date = deathRow['Date']
    
    dailyData = dailyData.append({
            'date': date,
            'country': 'Guinea',
            'new_cases': case,
            'new_deaths': death,
        }, ignore_index=True)
#set the column to the correct type
dailyData['new_cases'] = pd.to_numeric(dailyData['new_cases'])
dailyData['new_deaths'] = pd.to_numeric(dailyData['new_deaths'])
dailyData['date'] = pd.to_datetime(dailyData['date'])

#compute the average per year and add it to the dataframe
temp = dailyData.groupby(dailyData['date'].dt.year).sum().divide(dailyData.shape[0])
temp['country'] = 'Guinea'    
frames = [yearAverage, temp]
yearAverage = pd.concat(frames)

#### Liberia
Same logic as Guinea

We count as new case:
-  New case/s (confirmed)

We count as new death:
-  Newly reported deaths

In [322]:
#Daily data
dailyData = pd.DataFrame(columns=['date', 'country', 'new_cases', 'new_deaths'])

for filename in glob.glob(liberiaFolderPath + "/*.csv"):
    df = pd.read_csv(filename, index_col='Variable')
    
    casesRow = df.loc["New case/s (confirmed)"]
    case = casesRow['National']
    
    
    deathRow = df.loc["Newly reported deaths"]    
    death = deathRow['National']
    
    date = deathRow['Date']
    
    dailyData = dailyData.append({
            'date': date,
            'country': 'Liberia',
            'new_cases': case,
            'new_deaths': death,
        }, ignore_index=True)
#set the column to the correct type
dailyData['new_cases'] = pd.to_numeric(dailyData['new_cases'])
dailyData['new_deaths'] = pd.to_numeric(dailyData['new_deaths'])
dailyData['date'] = pd.to_datetime(dailyData['date'])

#compute the average per year and add it to the dataframe
temp = dailyData.groupby(dailyData['date'].dt.year).sum().divide(dailyData.shape[0])
temp['country'] = 'Liberia'
frames = [yearAverage, temp]
yearAverage = pd.concat(frames)

#### Sri Lanka
Same logic as Guinea and Liberia

We count as new case:
-  new_confirmed

We count as new death:
-  etc_new_deaths

In [323]:
#Daily data
dailyData = pd.DataFrame(columns=['date', 'country', 'new_cases', 'new_deaths'])

for filename in glob.glob(slFolderPath + "/*.csv"):
    df = pd.read_csv(filename, index_col='variable')
    
    casesRow = df.loc["new_confirmed"]
    case = casesRow['National']
    
    
    deathRow = df.loc["etc_new_deaths"]    
    death = deathRow['National']
    
    date = deathRow['date']
    
    dailyData = dailyData.append({
            'date': date,
            'country': 'Sri Lanka',
            'new_cases': case,
            'new_deaths': death,
        }, ignore_index=True)
#set the column to the correct type
dailyData['new_cases'] = pd.to_numeric(dailyData['new_cases'])
dailyData['new_deaths'] = pd.to_numeric(dailyData['new_deaths'])
dailyData['date'] = pd.to_datetime(dailyData['date'])

#compute the average per year and add it to the dataframe
temp = dailyData.groupby(dailyData['date'].dt.year).sum().divide(dailyData.shape[0])
temp['country'] = 'Sri Lanka'
frames = [yearAverage, temp]
yearAverage = pd.concat(frames)

#### Final result
The final resulte give the year as index and the daily average per year of new cases and new deaths for each country

In [324]:
yearAverage

,country,new_cases,new_deaths
2014,Guinea,12.545455,4.045455
2014,Liberia,176.600000,20.380000
2014,Sri Lanka,42.504854,1.262136


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here